<a href="https://colab.research.google.com/github/farhanfuadabir/NurseCareChallenge2020/blob/main/NursingActivity_DataPreparation_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Add** Google Drive


```
fuad.camera08@gmail.com
```



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# **Lab** Data

## **Import** and Process



*   Import label and data files
*   Check for nan
*   Filter duplicates
*   Convert to datetime


In [ ]:
import pandas as pd
import numpy as np

#----------------------  Import label and data files --------------------------- 

lab_label_path = '/content/drive/My Drive/Nursing Activity Dataset/Lab/label.csv'
lab_data_path = '/content/drive/My Drive/Nursing Activity Dataset/Lab/acc.csv'
#lab_label_path = '/content/drive/My Drive/nurse activity/Lab/labels_lab_2users.csv'
#lab_data_path = '/content/drive/My Drive/nurse activity/Lab/bigact_raw_lab_acc.csv'

print('Importing label file...', end='')
label = pd.read_csv(lab_label_path)
print('Done | Shape: ', label.shape)
print('Importing data file...', end='')
data = pd.read_csv(lab_data_path)
print('Done | Shape: ', data.shape)

print('\nImported raw files...')
print("\ndata:\n", data.head())
print("data shape: ", data.shape)
print("\nlabel:\n", label.head())
print("label shape: ", label.shape)

#----------------------------  Check for nan  ----------------------------------

if label.isna().any().any() == True:
  print('\nMissing value detected in label')
  # Drop nan rows
  print('Dropping nan rows...',end=' ')
  label.dropna(inplace=True)
  print('Done | Shape: ', label.shape)

if data.isna().any().any() == True:
  print('\nMissing value detected in data')
  # Drop nan rows
  print('Dropping nan rows...',end=' ')
  data.dropna(inplace=True)
  print('Done | Shape: ', data.shape)

#------------------------------ Filter duplicates ------------------------------

print('\nDroping duplicates and sorting...', end='')
label = label.drop_duplicates(keep='first').sort_values(by=['start']).reset_index(drop=True)
data = data.drop_duplicates(keep='first').sort_values(by=['datetime']).reset_index(drop=True)
print('Done | label shape: ', label.shape,' data shape: ', data.shape)

print("\n\nBefore datetime conversion...\n")
print("data:\n", data.head())
print("data shape: ", data.shape)
print("\nlabel:\n", label.head())
print("label shape: ", label.shape)


#------------------------  Convert to datetime  --------------------------------

label.finish = pd.to_datetime(label.finish) + pd.Timedelta('59s')
label.start = pd.to_datetime(label.start)

data_1 = data.loc[(data.datetime.str[-5:]=='+1000')].copy()
data_1.datetime = data_1.datetime.str[:-5]
data_1.datetime = pd.to_datetime(data_1.datetime) - pd.Timedelta('1h')

data_2 = data.loc[(data.datetime.str[-5:]=='+0900')].copy()
data_2.datetime = data_2.datetime.str[:-5]
data_2.datetime = pd.to_datetime(data_2.datetime)

data = pd.concat([data_1,data_2]).sort_index()

print("\n\nAfter datetime conversion...\n")
print("data:\n", data.head())
print("data shape: ", data.shape)
print("\nlabel:\n", label.head())
print("label shape: ", label.shape)


Importing label file...Done | Shape:  (423, 4)
Importing data file...Done | Shape:  (1064483, 5)

Imported raw files...

data:
    user_id                      datetime      x    y    z
0        1  2018-07-25T14:58:46.247+1000  0.612  7.7  0.0
1        1  2018-07-25T14:55:30.396+1000  5.286  7.7  0.0
2        1  2018-07-25T14:55:30.402+1000  5.286  7.7  0.0
3        1  2018-07-25T14:58:46.247+1000  0.612  7.7  0.0
4        1  2018-07-25T14:55:30.396+1000  5.286  7.7  0.0
data shape:  (1064483, 5)

label:
    user_id  act_id            start           finish
0       19       1  8/22/2018 11:06  8/22/2018 11:06
1       19       1  8/22/2018 11:18  8/22/2018 11:18
2       19       1  8/22/2018 11:19  8/22/2018 11:19
3        1       1  7/25/2018 12:24  7/25/2018 12:24
4        1       1  7/25/2018 12:26  7/25/2018 12:27
label shape:  (423, 4)

Droping duplicates and sorting...Done | label shape:  (386, 4)  data shape:  (617479, 5)


Before datetime conversion...

data:
    user_id        

### Datetime Conversion Analysis for 2018-07-25

> **Issue:** All other timestamp had timezone of +0900 and this date had timezone of +1000.

> **Observation:** The manual timestamp matching seemed to make more sense for +1 which is theoretically wrong but the analysis showed otherwise.

| Date | User | Adjustment | Data on the Day | Total Data | act_1 | act_2 | act_3 | act_4 | act_5 | act_6 | act_7 | act_8 | act_9 | act_10 | act_11 | act_12|
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---:| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 2018-07-25 | 01 | +1 hr | 60343 | 185552 | 5847 | act_2 | 7425 | 7460| 3815 | 5444 | 5397 | 4761 | 3399 | 5150 | 2838 | 1260 |
| 2018-07-25 | 01 | None | 29644 | 154853 | 2623 | 430 | -- | -- | 2665 | 2699 | 3131 | 2720 | 2689 | 5226 | 3062 | 4399|
| 2018-07-25 | 01 | -1 hr | 63366 | 188575 | 5629 | 7489 | 7447 | 7116 | 5216 | 5113 | 5381 | 4521 | 3333 | 5042 | 2913 | 4166 |
     

## Data **Labelling** and Post-**Process**


*   Find data within the timestamps of the *label.csv* file and label accordingly
*   Make separate dataframes
*   Write joblib and csv files


In [ ]:
#-------------------------- Adjust data and label ------------------------------
data_new = pd.DataFrame()
take_count = 0
print('Adjusting data and label...', end='')

for i in label.index:
  mask = (label.loc[i,:].start <= data.datetime) & (data.datetime <= label.loc[i,:].finish) & (data.user_id == label.loc[i,:].user_id)
  part = data.loc[mask].assign(take=take_count, label=label.loc[i,:].act_id)
  data_new = data_new.append(part,ignore_index=True)
  if (part.empty == False):
    take_count = take_count + 1
  
  

print('Done | shape: ', data_new.shape, end='\n\n')

print('Processed data info...\n')
print('Shape: ', data_new.shape)
print('columns: ', data_new.columns, end='\n\n')
print(data_new.user_id.value_counts().sort_index(), end='\n\n')
print(data_new.label.value_counts().sort_index())

display(data_new)

Adjusting data and label...Done | shape:  (188575, 7)

Processed data info...

Shape:  (188575, 7)
columns:  Index(['user_id', 'datetime', 'x', 'y', 'z', 'take', 'label'], dtype='object')

1      63366
19    125209
Name: user_id, dtype: int64

1     13600
2     19498
3     21305
4     24502
5     13597
6     16225
7     13490
8     15491
9     13628
10    10782
11    12046
12    14411
Name: label, dtype: int64


,user_id,datetime,x,y,z,take,label
0,1,2018-07-25 12:24:00.167,-1.494,8.925,0.550,0,1
1,1,2018-07-25 12:24:00.368,-1.417,8.964,0.549,0,1
2,1,2018-07-25 12:24:00.377,-1.417,8.964,0.549,0,1
3,1,2018-07-25 12:24:00.569,-1.379,9.040,0.542,0,1
4,1,2018-07-25 12:24:00.577,-1.379,9.040,0.542,0,1
...,...,...,...,...,...,...,...
188570,19,2018-08-23 15:15:58.319,-1.992,9.079,0.089,316,12
188571,19,2018-08-23 15:15:58.520,-2.758,9.347,0.247,316,12
188572,19,2018-08-23 15:15:58.543,-2.758,9.347,0.247,316,12
188573,19,2018-08-23 15:15:58.918,-2.949,8.657,0.321,316,12


In [ ]:
display(data_new['take'].unique())
print(data_new.shape)
data_new = data_new.append(pd.DataFrame(),ignore_index=True)
print(data_new.shape)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 171, 172, 173,
       179, 180, 182, 183, 185, 187, 189, 192, 201, 202, 203, 20

(188575, 7)
(188575, 7)


In [ ]:
#------------------------- Make separate dataframes ----------------------------

data_new = data_new.reindex(columns=['user_id','datetime','take','x','y','z','label'])
label_new = data_new.label
data_newX = data_new.drop(['y','z','label'],axis=1)
data_newY = data_new.drop(['x','z','label'],axis=1)
data_newZ = data_new.drop(['x','y','label'],axis=1)

display(label_new)
display(data_newX)
display(data_newY)
display(data_newZ)

0          1
1          1
2          1
3          1
4          1
          ..
188570    12
188571    12
188572    12
188573    12
188574    12
Name: label, Length: 188575, dtype: int64

,user_id,datetime,take,x
0,1,2018-07-25 12:24:00.167,0,-1.494
1,1,2018-07-25 12:24:00.368,0,-1.417
2,1,2018-07-25 12:24:00.377,0,-1.417
3,1,2018-07-25 12:24:00.569,0,-1.379
4,1,2018-07-25 12:24:00.577,0,-1.379
...,...,...,...,...
188570,19,2018-08-23 15:15:58.319,385,-1.992
188571,19,2018-08-23 15:15:58.520,385,-2.758
188572,19,2018-08-23 15:15:58.543,385,-2.758
188573,19,2018-08-23 15:15:58.918,385,-2.949


,user_id,datetime,take,y
0,1,2018-07-25 12:24:00.167,0,8.925
1,1,2018-07-25 12:24:00.368,0,8.964
2,1,2018-07-25 12:24:00.377,0,8.964
3,1,2018-07-25 12:24:00.569,0,9.040
4,1,2018-07-25 12:24:00.577,0,9.040
...,...,...,...,...
188570,19,2018-08-23 15:15:58.319,385,9.079
188571,19,2018-08-23 15:15:58.520,385,9.347
188572,19,2018-08-23 15:15:58.543,385,9.347
188573,19,2018-08-23 15:15:58.918,385,8.657


,user_id,datetime,take,z
0,1,2018-07-25 12:24:00.167,0,0.550
1,1,2018-07-25 12:24:00.368,0,0.549
2,1,2018-07-25 12:24:00.377,0,0.549
3,1,2018-07-25 12:24:00.569,0,0.542
4,1,2018-07-25 12:24:00.577,0,0.542
...,...,...,...,...
188570,19,2018-08-23 15:15:58.319,385,0.089
188571,19,2018-08-23 15:15:58.520,385,0.247
188572,19,2018-08-23 15:15:58.543,385,0.247
188573,19,2018-08-23 15:15:58.918,385,0.321


In [ ]:
#----------------------------- Export csv file ---------------------------------

from joblib import dump

export_path = '/content/drive/My Drive/Nursing Activity Dataset/Lab/'

#print('Exporting joblib at: ', export_path + 'data_processed.joblib', '...', end='')
#dump(data_new, export_path + 'data_processed.joblib')
#print('Done')

print('Exporting csv at: ', export_path + 'data_processed.csv', '...', end='')
data_new.to_csv(export_path + 'data_processed.csv', index=False)
print('Done')


Exporting csv at:  /content/drive/My Drive/Nursing Activity Dataset/Lab/data_processed.csv ...Done


In [ ]:
#--------------------------- Export Individual Channels ------------------------

from joblib import dump

export_path = '/content/drive/My Drive/Nursing Activity Dataset/Lab/'

# X axis
print('Exporting joblib at: ', export_path + 'Acc_x.joblib', '...', end='')
dump(data_newX, export_path + 'Acc_x.joblib')
print('Done')

# Y axis
print('Exporting joblib at: ', export_path + 'Acc_y.joblib', '...', end='')
dump(data_newY, export_path + 'Acc_y.joblib')
print('Done')

# Z axis
print('Exporting joblib at: ', export_path + 'Acc_z.joblib', '...', end='')
dump(data_newZ, export_path + 'Acc_z.joblib')
print('Done')

# label
print('Exporting joblib at: ', export_path + 'Label.joblib', '...', end='')
dump(label_new, export_path + 'Label.joblib')
print('Done')



Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Lab/Acc_x.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Lab/Acc_y.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Lab/Acc_z.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Lab/Label.joblib ...Done


# Clear **RAM**

In [ ]:
#--------------------------------- Reset Variables -----------------------------

%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
#----------------------------------- Clear RAM ---------------------------------

import gc
gc.collect()

15

In [ ]:
#-------------------------- Delete a certain variable --------------------------

# del <variable name>
#del data_new
#del label

# **Field** Data

## **Import** and Process



*   Import label and data files
*   Check for nan
*   Filter duplicates
*   Convert to datetime


In [ ]:
import pandas as pd
import numpy as np


#----------------------  Import label and data files --------------------------- 

field_label_path = '/content/drive/My Drive/Nursing Activity Dataset/Field/label.csv'
field_data_path = '/content/drive/My Drive/Nursing Activity Dataset/Field/'
field_data_files = ['acc_user04.csv', 'acc_user07.csv', 'acc_user08.csv', 
                    'acc_user18.csv', 'acc_user38.csv', 'acc_user51.csv']
# field_data_files = ['acc_user38.csv', 'acc_user51.csv']

print('Importing label file...', end='')
label = pd.read_csv(field_label_path)
label = label.drop_duplicates(keep='first').sort_values(by=['start']).reset_index(drop=True)
label.start = label.start.str[:-5]
label.start = pd.to_datetime(label.start)
label.finish = label.finish.str[:-5]
label.finish = pd.to_datetime(label.finish) + pd.Timedelta('999millis')
label = label.loc[label.start < label.finish]
print('Done | Shape: ', label.shape)


def import_process(path, filenames):
  
  print('Importing ' + user + ' file...', end='')
  temp_data = pd.read_csv(path + user)
  temp_data = temp_data.drop_duplicates(keep='first')
  temp_data.datetime = temp_data.datetime.str[:-5]
  temp_data.datetime = pd.to_datetime(temp_data.datetime)
  print('Done | Shape: ', temp_data.shape)

  return temp_data

data = pd.DataFrame()

for user in field_data_files:
  data = data.append(import_process(field_data_path, user), ignore_index=True)

data = data.sort_values(by=['datetime']).reset_index(drop=True)

print('\nImported raw files...')
print("\ndata:\n")
display(data.head())
print("data shape: ", data.shape)
print("\nlabel:\n")
display(label.head())
print("label shape: ", label.shape)

#----------------------------  Check for nan  ----------------------------------

if label.isna().any().any() == True:
  print('\nMissing value detected in label')
  # Drop nan rows
  print('Dropping nan rows...',end=' ') 
  label.dropna(inplace=True)
  print('Done | Shape: ', label.shape)

if data.isna().any().any() == True:
  print('\nMissing value detected in data')
  # Drop nan rows
  print('Dropping nan rows...',end=' ')
  data.dropna(inplace=True)
  print('Done | Shape: ', data.shape)


Importing label file...Done | Shape:  (1466, 4)
Importing acc_user04.csv file...Done | Shape:  (8720962, 5)
Importing acc_user07.csv file...Done | Shape:  (14019561, 5)
Importing acc_user08.csv file...Done | Shape:  (11525126, 5)
Importing acc_user18.csv file...Done | Shape:  (9022029, 5)
Importing acc_user38.csv file...Done | Shape:  (9250951, 5)
Importing acc_user51.csv file...Done | Shape:  (6972779, 5)

Imported raw files...

data:



,user_id,datetime,x,y,z
0,8,2018-11-30 07:07:34.106,-9.653,-0.306,0.055
1,8,2018-11-30 07:07:34.303,-9.653,-0.306,0.057
2,8,2018-11-30 07:07:34.309,-9.653,-0.306,0.057
3,8,2018-11-30 07:07:34.502,-9.692,-0.268,0.056
4,8,2018-11-30 07:07:34.508,-9.692,-0.268,0.056


data shape:  (59511408, 5)

label:



,user_id,activity_type_id,start,finish
0,38,1,2018-11-21 14:15:29,2018-11-21 14:15:31.999
1,51,1,2018-11-21 14:15:35,2018-11-21 14:15:38.999
2,51,3,2018-11-21 14:31:31,2018-11-21 14:31:32.999
3,51,5,2018-11-21 14:31:52,2018-11-21 14:31:54.999
4,18,5,2018-11-22 19:17:46,2018-11-22 19:19:29.999


label shape:  (1466, 4)


## Data **Labelling** and Post-**Process**


*   Find data within the timestamps of the *label.csv* file and label accordingly
*   Make separate dataframes
*   Write joblib and csv files


In [ ]:
#-------------------------- Adjust data and label ------------------------------
data_new = pd.DataFrame()
take_count = 0
print('Adjusting data and label...', end='')

for i in label.index:
  mask = (label.loc[i,:].start <= data.datetime) & (data.datetime <= label.loc[i,:].finish) & (data.user_id == label.loc[i,:].user_id)
  part = data.loc[mask].assign(take=take_count, label=label.loc[i,:].activity_type_id)
  data_new = data_new.append(part,ignore_index=True)
  if (part.empty == False):
    take_count = take_count + 1

print('Done | shape: ', data_new.shape, end='\n\n')

print('Processed data info...\n')
print('Shape: ', data_new.shape)
print('columns: ', data_new.columns, end='\n\n')
print(data_new.user_id.value_counts().sort_index(), end='\n\n')
print(data_new.label.value_counts().sort_index())

display(data_new)

Adjusting data and label...Done | shape:  (1321144, 7)

Processed data info...

Shape:  (1321144, 7)
columns:  Index(['user_id', 'datetime', 'x', 'y', 'z', 'take', 'label'], dtype='object')

4     414208
7     149195
8      22664
18    715218
38       845
51     19014
Name: user_id, dtype: int64

1      18406
2       2448
3      61749
4     303383
5     806544
6      14201
7      77145
9       2067
10      1881
12     33320
Name: label, dtype: int64


,user_id,datetime,x,y,z,take,label
0,18,2018-12-01 22:50:30.481,-5.171,7.278,3.390,0,9
1,18,2018-12-01 22:50:30.497,-3.754,8.274,3.373,0,9
2,18,2018-12-01 22:50:30.506,-1.072,8.810,3.633,0,9
3,18,2018-12-01 22:50:30.511,-1.072,8.810,3.633,0,9
4,18,2018-12-01 22:50:30.526,-0.574,8.351,3.849,0,9
...,...,...,...,...,...,...,...
1321139,8,2019-01-01 06:30:32.527,-0.842,8.198,4.768,963,12
1321140,8,2019-01-01 06:30:32.685,-1.110,8.236,4.942,963,12
1321141,8,2019-01-01 06:30:32.689,-1.110,8.236,4.942,963,12
1321142,8,2019-01-01 06:30:32.886,-0.804,8.198,4.677,963,12


In [ ]:
#------------------------- Make separate dataframes ----------------------------

data_new = data_new.reindex(columns=['user_id','datetime','take','x','y','z','label'])
label_new = data_new.label
data_newX = data_new.drop(['y','z','label'],axis=1)
data_newY = data_new.drop(['x','z','label'],axis=1)
data_newZ = data_new.drop(['x','y','label'],axis=1)

display(label_new)
display(data_newX)
display(data_newY)
display(data_newZ)

0           9
1           9
2           9
3           9
4           9
           ..
1321139    12
1321140    12
1321141    12
1321142    12
1321143    12
Name: label, Length: 1321144, dtype: int64

,user_id,datetime,take,x
0,18,2018-12-01 22:50:30.481,0,-5.171
1,18,2018-12-01 22:50:30.497,0,-3.754
2,18,2018-12-01 22:50:30.506,0,-1.072
3,18,2018-12-01 22:50:30.511,0,-1.072
4,18,2018-12-01 22:50:30.526,0,-0.574
...,...,...,...,...
1321139,8,2019-01-01 06:30:32.527,963,-0.842
1321140,8,2019-01-01 06:30:32.685,963,-1.110
1321141,8,2019-01-01 06:30:32.689,963,-1.110
1321142,8,2019-01-01 06:30:32.886,963,-0.804


,user_id,datetime,take,y
0,18,2018-12-01 22:50:30.481,0,7.278
1,18,2018-12-01 22:50:30.497,0,8.274
2,18,2018-12-01 22:50:30.506,0,8.810
3,18,2018-12-01 22:50:30.511,0,8.810
4,18,2018-12-01 22:50:30.526,0,8.351
...,...,...,...,...
1321139,8,2019-01-01 06:30:32.527,963,8.198
1321140,8,2019-01-01 06:30:32.685,963,8.236
1321141,8,2019-01-01 06:30:32.689,963,8.236
1321142,8,2019-01-01 06:30:32.886,963,8.198


,user_id,datetime,take,z
0,18,2018-12-01 22:50:30.481,0,3.390
1,18,2018-12-01 22:50:30.497,0,3.373
2,18,2018-12-01 22:50:30.506,0,3.633
3,18,2018-12-01 22:50:30.511,0,3.633
4,18,2018-12-01 22:50:30.526,0,3.849
...,...,...,...,...
1321139,8,2019-01-01 06:30:32.527,963,4.768
1321140,8,2019-01-01 06:30:32.685,963,4.942
1321141,8,2019-01-01 06:30:32.689,963,4.942
1321142,8,2019-01-01 06:30:32.886,963,4.677


In [ ]:
#----------------------------- Export csv file ---------------------------------

from joblib import dump

export_path = '/content/drive/My Drive/Nursing Activity Dataset/Field/'

#print('Exporting joblib at: ', export_path + 'data_processed.joblib', '...', end='')
#dump(data_new, export_path + 'data_processed.joblib')
#print('Done')

print('Exporting csv at: ', export_path + 'data_processed.csv', '...', end='')
data_new.to_csv(export_path + 'data_processed.csv', index=False)
print('Done')


Exporting csv at:  /content/drive/My Drive/Nursing Activity Dataset/Field/data_processed.csv ...Done


In [ ]:
#--------------------------- Export Individual Channels ------------------------

from joblib import dump

export_path = '/content/drive/My Drive/Nursing Activity Dataset/Field/'

# X axis
print('Exporting joblib at: ', export_path + 'Acc_x.joblib', '...', end='')
dump(data_newX, export_path + 'Acc_x.joblib')
print('Done')

# Y axis
print('Exporting joblib at: ', export_path + 'Acc_y.joblib', '...', end='')
dump(data_newY, export_path + 'Acc_y.joblib')
print('Done')

# Z axis
print('Exporting joblib at: ', export_path + 'Acc_z.joblib', '...', end='')
dump(data_newZ, export_path + 'Acc_z.joblib')
print('Done')

# label
print('Exporting joblib at: ', export_path + 'Label.joblib', '...', end='')
dump(label_new, export_path + 'Label.joblib')
print('Done')



Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Field/Acc_x.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Field/Acc_y.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Field/Acc_z.joblib ...Done
Exporting joblib at:  /content/drive/My Drive/Nursing Activity Dataset/Field/Label.joblib ...Done


# Miscellaneous

In [ ]:
import numpy as np
import pandas as pd
from joblib import load, dump

path_lab = '/content/drive/My Drive/NCC Dataset 2020/Train/Lab/Label_new.joblib'
path_lab_new = '/content/drive/My Drive/NCC Dataset 2020/Train/Lab/Label.joblib'
path_field = '/content/drive/My Drive/NCC Dataset 2020/Train/Field/Label_new.joblib'
path_field_new = '/content/drive/My Drive/NCC Dataset 2020/Train/Field/Label.joblib'

df = load(path_field)
display(df)